## Bokeh and Declarative Widgets Examples

The **[Bokeh]()** interactive visualization library integrates well with **Jupyter Declarative Widgets**. Declarative Widgets bring the interactivity of Polymer to the feature-rich visualizations provided by Bokeh; when combined the two tools offer powerful notebook capabilities.

This notebook shows some ways that Bokeh and Declarative Widgets can be used together.

-----------------

#### Setup

First we need to install `bokeh`, initialize the declarative widgets extension, import the libraries and elements that we'll use, and tell `bokeh` to output using the notebook environment:

In [ ]:
!conda install -y bokeh=0.10*

In [ ]:
from bokeh.models import ColumnDataSource, CustomJS
from bokeh.plotting import figure, show, output_notebook
from urth import widgets

import numpy as np

widgets.init()

In [ ]:
%%html
<!-- Import dependencies -->
<link rel='import' href='urth_components/paper-slider/paper-slider.html'
        is='urth-core-import' package='PolymerElements/paper-slider'>
<link rel="import" href="urth_components/urth-viz-table/urth-viz-table.html" 
        is="urth-core-import">
    
<!-- Define data channel -->
<urth-core-channel name='table' id='tableChannel'></urth-core-channel>

In [ ]:
output_notebook()

-----------------

### Dynamic Plotting: Random Data Generator

This example shows how the `urth-core-function` can be used to dynamically update a Bokeh plot based on user input. 

Below, we create the Bokeh plot. Notice how it uses `source`, which will be used for data updates:

In [ ]:
source = ColumnDataSource(data=dict(x=[], y=[]))

p = figure(title="Random Gaussian Data", 
           plot_height=300, 
           plot_width=600, 
           y_range=[-50, 50])
p.circle('x', 'y', source=source);

The `generate` function generates random data points using the parameters from the sliders, and updates the plot using Bokeh's `push_notebook` function.

`show_ex1` is used to display the plot created above:

In [ ]:
def generate(mean=0.0, std=1.0, n=100):
    x = np.arange(0, n, 1)
    y = np.random.normal(mean, std, size=n)
    source.data['x'] = x
    source.data['y'] = y
    source.push_notebook()
    
def show_ex1():
    show(p)

Now we declare the widgets and Polymer elements:

* `urth-core-function` bound to `show_ex1` with `auto` to display the plot
* `urth-core-function` bound to `generate` to generate data and update the plot
* `paper-slider` elements to adjust the arguments to `generate`

In [ ]:
%%html
<urth-core-function ref="show_ex1" auto></urth-core-function>
<template is="dom-bind">
    <urth-core-function ref="generate" arg-mean="{{mean}}" arg-std="{{std}}" arg-n="{{n}}" auto></urth-core-function>
    <paper-slider min="-10" max="10" step="0.5" value="{{mean}}"></paper-slider>mean <br/>
    <paper-slider min="0" max="10" step="0.5" value="{{std}}"></paper-slider>std dev<br/>
    <paper-slider min="0" max="1000" step="100" value="{{n}}"></paper-slider>number of points<br/>
</template>

-----------------

### Events and Selections: Selected Data Table

We often want to further investigate data points based on a visualization. For instance, we may want to look at the data records of outliers on a scatter plot, or more generally the records for any subset of points in the plot.

This example shows how `urth-core-channel` and `urth-core-bind` can be used to dynamically populate an `urth-viz-table` using points that a user selects on a Bokeh plot.

First we declare the Bokeh plot with some data: 

In [ ]:
xs = list(np.arange(-0.7, 0.8, 0.1)) + [-0.25, 0.25]
ys = [x ** 2 for x in xs[:-2]] + [0.8, 0.8]

source2 = ColumnDataSource(data=dict(x=xs, y=ys))
f1 = figure(tools='tap,box_select,lasso_select')
f1.circle('x', 'y', source=source2, radius=0.05);

Then we attach a callback that is fired when points on the plot are selected.

We define the callback to set the table data with the currently selected points by setting `data` on the `table` channel:

In [ ]:
# setup callback
source2.callback = CustomJS(code=""" 
    var selectedIndices = cb_obj.get('selected')['1d'].indices;
    var allData = cb_obj.get('data');
    var rowData = [];
    for (var i = 0; i < selectedIndices.length; i++){
        var row = [];
        row.push(allData['x'][selectedIndices[i]]);
        row.push(allData['y'][selectedIndices[i]]);
        rowData.push(row);
    }
    var tableData = {
        data: rowData,
        columns: ['x', 'y']
    }
    
    document.getElementById('tableChannel').set('data', tableData);
""")

In [ ]:
def show_ex2():
    show(f1)

Now we declare the widgets and Polymer elements:

* `urth-core-function` bound to `show_ex2` with `auto` to display the plot
* `urth-core-bind` template using the `table` channel
* `urth-viz-table` and `urth-viz-col`s

In [ ]:
%%html
<urth-core-function ref="show_ex2" auto></urth-core-function>
<template is='urth-core-bind' channel='table'>
    <h4>Selected Points</h4>
    <urth-viz-table datarows="{{ data.data }}" selection="{{sel}}" columns="{{ data.columns }}" rows-visible=6>
        <urth-viz-col></urth-viz-col>
        <urth-viz-col></urth-viz-col>
    </urth-viz-table>
    <p>Use the selection tools in the upper-right corner to select points on the plot:</p>
</template>

------------------------

### Streaming Plots with Spark

In this example, we display streaming data from PySpark on a Bokeh plot, and use `urth-core-function`s to build a small interface for controlling the stream.

In [ ]:
import pandas as pd
import numpy as np
import pyspark
from pyspark.streaming import StreamingContext

sc = pyspark.SparkContext('local[*]')
sqlContext = pyspark.SQLContext(sc)

First we create a `stream` containing the data points from `xs` and `ys`:

In [ ]:
xs = np.arange(0, 100, 1)
ys = np.random.random_integers(low=0, high=10, size=len(xs))

def create_rdd(x, y):
    rdd = sc.parallelize([x, y])
    return rdd

queue = [create_rdd(x, y) for x, y in zip(xs, ys)]
streaming_context = StreamingContext(sc, 1)
stream = streaming_context.queueStream(queue)

And a function that updates the plot when a new data point arrives:

In [ ]:
def update_plot(rdd):
    data = rdd.collect()
    if data:
        source3.data['x'] = list(source3.data['x']) + [data[0]]
        source3.data['y'] = list(source3.data['y']) + [data[1]]
        source3.push_notebook()

In [ ]:
stream.foreachRDD(update_plot)

Then we declare the Bokeh plot. Notice how `source3` is used below and in `update_plot`:

In [ ]:
source3 = ColumnDataSource(data=dict(x=[], y=[]))
f2 = figure(y_range=[0, 10])
f2.line('x', 'y', source=source3);

Then we make functions that we'll bind to:

In [ ]:
def show_ex3():
    show(f2)
    
def start_stream():
    streaming_context.start()

def stop_stream():
    streaming_context.stop()

Finally, we declare the widgets and Polymer elements:

* `urth-core-function` bound to `show_ex3` with `auto` to display the plot
* `urth-core-function` bound to `start_stream`
* `urth-core-function` bound to `stop_stream`

In [ ]:
%%html
<urth-core-function ref="show_ex3" auto></urth-core-function>
<urth-core-function id="start" ref="start_stream"></urth-core-function>
<urth-core-function id="stop" ref="stop_stream"></urth-core-function>
<button onclick="$('#start')[0].invoke()">Start</button>
<button onclick="$('#stop')[0].invoke();">Stop</button>